In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomCNN(nn.Module):
    def __init__(self, input_channels:int, nodes:list, kernels:list, num_classes:int=10):
        """
        input channels is the number of channels in the input data (e.g., 1 for grayscale, 3 for RGB)
        nodes is a list of nodes per layer i.e. 128, 64, 32
 kernals is a list of the kernals used per layer
 number of classes is thenumber of outputs (the number of emotions to possibly classify)
 this creates the neural network with the specified layers and the kernals
        """
        super(CustomCNN, self).__init__()

        assert len(nodes) == len(kernels), "nodes and kernels must have the same length"

        layers = []
        in_channels = input_channels

        for out_channels, k in zip(nodes, kernels):
            conv = nn.Conv2d(in_channels, out_channels, kernel_size=k, padding=k//2)
            layers.append(conv)
            layers.append(nn.ReLU())
            layers.append(nn.MaxPool2d(2))  
            in_channels = out_channels

        self.conv_layers = nn.Sequential(*layers)

        
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(nodes[-1], num_classes)  

    def forward(self, x):
        x = self.conv_layers(x)
        x = F.adaptive_avg_pool2d(x, (1, 1))  
        x = self.flatten(x)
        x = self.fc(x)
        return x


: 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import json
import tvm
from tvm import relay
from tvm.contrib import graph_executor

class CustomCNN(nn.Module):
    def __init__(self, input_channels:int, nodes:list, kernels:list, num_classes:int=10):
        super(CustomCNN, self).__init__()
        assert len(nodes) == len(kernels), "nodes and kernels must have the same length"

        layers = []
        in_channels = input_channels
        for out_channels, k in zip(nodes, kernels):
            conv = nn.Conv2d(in_channels, out_channels, kernel_size=k, padding=k//2)
            layers += [conv, nn.ReLU(), nn.MaxPool2d(2)]
            in_channels = out_channels

        self.conv_layers = nn.Sequential(*layers)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(nodes[-1], num_classes)

    def forward(self, x):
        x = self.conv_layers(x)
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = self.flatten(x)
        x = self.fc(x)
        return x


# ==============================
# SAVE / LOAD FUNCTIONS
# ==============================

def save_customcnn(model, filename="customcnn.pth", infofile="customcnn_info.txt", **meta):
    """Save model weights + architecture info in same folder."""
    torch.save(model.state_dict(), filename)
    info = {
        "input_channels": meta.get("input_channels"),
        "nodes": meta.get("nodes"),
        "kernels": meta.get("kernels"),
        "num_classes": meta.get("num_classes"),
    }
    with open(infofile, "w") as f:
        f.write(json.dumps(info))
    print(f"✅ Model saved as {filename} with metadata {infofile}")


def load_customcnn(weightfile="customcnn.pth", infofile="customcnn_info.txt"):
    """Reload a CustomCNN using saved metadata + weights."""
    with open(infofile, "r") as f:
        info = json.loads(f.read())
    model = CustomCNN(
        input_channels=info["input_channels"],
        nodes=info["nodes"],
        kernels=info["kernels"],
        num_classes=info["num_classes"],
    )
    model.load_state_dict(torch.load(weightfile, map_location=torch.device("cpu")))
    model.eval()
    print(f"✅ Model loaded from {weightfile}")
    return model


# ==============================
# TVM CONVERSION FUNCTION
# ==============================

def export_to_tvm(model, input_shape=(1, 3, 64, 64), target="llvm", output_prefix="tvm_model"):
    """Convert a PyTorch CNN to a TVM deployable format."""
    model.eval()
    example_input = torch.randn(input_shape)

    # 1. Trace PyTorch model
    traced = torch.jit.trace(model, example_input)

    # 2. Convert to TVM Relay
    shape_list = [("input0", example_input.shape)]
    mod, params = relay.frontend.from_pytorch(traced, shape_list)

    # 3. Build TVM module
    with tvm.transform.PassContext(opt_level=3):
        lib = relay.build(mod, target=target, params=params)

    # 4. Save compiled outputs
    lib.export_library(f"{output_prefix}.so")
    with open(f"{output_prefix}.json", "w") as f_json:
        f_json.write(lib.graph_json)
    with open(f"{output_prefix}.params", "wb") as f_params:
        f_params.write(relay.save_param_dict(lib.params))

    print(f"✅ TVM module exported: {output_prefix}.so / .json / .params")
    return lib


In [ ]:

model = CustomCNN(
    input_channels=3,
    nodes=[32, 64, 128],
    kernels=[3, 3, 5],
    num_classes=10
)

print(model)
